In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from yass.configuration import Configs
from yass.preprocess import Preprocessor
from yass.neuralnet import NeuralNetDetector
from yass.mainprocess import Mainprocessor
from yass.deconvolution import Deconvolution
import numpy as np

%matplotlib inline
import matplotlib.pyplot as plt

In [7]:
cfg = Configs(root='/Users/edu/Downloads/yass')

Run filtering, standardizing, nn detection, getting and saving waveforms

In [9]:
pp = Preprocessor(cfg)
score, clr_idx, spt, bfCounter = pp.process()

/Users/edu/dev/grossman/YASS/src/yass/preprocess.py:601: ComplexWarning: Casting complex values to real discards the imaginary part
  return rot
/Users/edu/dev/grossman/YASS/src/yass/preprocess.py:598: ComplexWarning: Casting complex values to real discards the imaginary part
  w, v = np.linalg.eig(ss[:,:,c])


In [10]:
len(score)

49

In [11]:
clr_idx

[array([   0,    1,    2, ..., 1014, 1015, 1016]),
 array([   0,    1,    2, ..., 1109, 1110, 1111]),
 array([  0,   1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,
         13,  14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,
         26,  27,  28,  29,  30,  31,  32,  33,  34,  35,  36,  37,  38,
         39,  40,  41,  42,  43,  44,  45,  46,  47,  48,  49,  50,  51,
         52,  53,  54,  55,  56,  57,  58,  59,  60,  61,  62,  63,  64,
         65,  66,  67,  68,  69,  70,  71,  72,  73,  74,  75,  76,  77,
         78,  79,  80,  81,  82,  83,  84,  85,  86,  87,  88,  89,  90,
         91,  92,  93,  94,  95,  96,  97,  98,  99, 100, 101, 102, 103]),
 array([], dtype=int64),
 array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
        17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33,
        34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50,
        51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63,

In [12]:
spt[0]

array([   5331,    5995,   18619, ..., 5964590, 5967721, 5995356], dtype=int32)

In [13]:
bfCounter

array([[        0,         0,   3111003],
       [        1,   3111003,   3401608],
       [        2,   6512611,    318136],
       [        4,   6830747,    278369],
       [        5,   7109116,   7623028],
       [        6,  14732144,    691334],
       [        7,  15423478,    296723],
       [        8,  15720201,    180481],
       [        9,  15900682,   7476196],
       [       10,  23376878,  22973090],
       [       11,  46349968,   3807328],
       [       12,  50157296,   4581738],
       [       13,  54739034,    473984],
       [       14,  55213018,    144900],
       [       15,  55357918,   1105104],
       [       16,  56463022,  16942352],
       [       17,  73405374,    391552],
       [       18,  73796926,    486864],
       [       19,  74283790,   1736385],
       [       20,  76020175,    528885],
       [       21,  76549060,   2588880],
       [       22,  79137940,    123165],
       [       23,  79261105,   1411004],
       [       24,  80672109,   11

Run Triage, Coreset, Masking, Clustering, Template Merging

In [9]:
mp = Mainprocessor(cfg,score,clr_idx,bfCounter,spt)
mp.mainProcess()

ValueError: a must be non-empty

Check these good looking templates!

In [ ]:
plt.plot(mp.templates[:,:,61].T)
plt.show()

Running Deconvolution (not fully put into the pipeline yet..)

In [ ]:
C,R,K = mp.templates.shape
neighchan_big = mp.config.n_steps_neigh_channels(2)
mp.open_wfFile()

sts=np.zeros(10000000,'int32')
ids=np.zeros(10000000,'int32')

count = 0
for c in range(C):
    print(c)
    templates = np.transpose(mp.templates[neighchan_big[c]],(1,0,2))
    k_keep = np.sum(np.abs(templates),axis=(0,1)) > 0
    templates = templates[:,:,k_keep]
    dc = Deconvolution(cfg, templates, mp.load_wf(c), mp.spt[c])
    dc.set_templates()
    st, idd = dc.fullMPMU()

    ndata = st.shape[0]
    if np.sum(k_keep) == K:
        sts[count:(count+ndata)] = st
        ids[count:(count+ndata)] = idd
    else:
        idd_temp = np.zeros(ndata,'int32')
        k_idx = np.where(k_keep)[0]
        for j in range(ndata):
            idd_temp[j] = k_idx[idd[j]]
        sts[count:(count+ndata)] = st
        ids[count:(count+ndata)] = idd_temp
    count += ndata

sts = sts[:count]
ids = ids[:count]